# Chatbot with LangChain conversational chain and OpenAI 🤖💬

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/generative-ai/chatbot.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll build a chatbot that can respond to questions about custom data, such as policies of an employer.

The chatbot uses LangChain's `ConversationalRetrievalChain` and has the following capabilities:
- Answer questions asked in natural language
- Run hybrid search in Elasticsearch to find documents that answer the question
- Extract and summarize the answer using OpenAI LLM
- Maintain conversational memory for follow-up questions

## Requirements 🧰
For this example, you will need:

- Python 3.6 or later
- An Elastic deployment
  - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration))
- OpenAI account

### Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, follow these steps to create one.
1. Go to https://cloud.elastic.co/registration and sign up for a free trial
2. Select **Create Deployment** and follow the instructions

## Install packages 📦

First we `pip install` the packages we need for this example.

In [ ]:
%pip install -U langchain==0.0.245 openai elasticsearch

## Initialize clients 🔌

Next we input credentials with `getpass`. `getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [ ]:
from getpass import getpass

ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_USERNAME = getpass("Elastic username: ")
ELASTIC_PASSWORD = getpass("Elastic password: ")
OPENAI_API_KEY = getpass("OpenAI API key: ")

With these credentials we can now initialize the Elasticsearch Python client.

In [ ]:
from elasticsearch import Elasticsearch

elasticsearch_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD)
)

print("Connected to Elasticsearch\n", elasticsearch_client.info())

Connected to Elasticsearch
 {'name': 'instance-0000000000', 'cluster_name': 'dacecbd336e6450d882c8adef9d131eb', 'cluster_uuid': 'PIWjifvgTcW7iV5zJmu0PA', 'version': {'number': '8.9.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d', 'build_date': '2023-07-19T14:43:58.555259655Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Create index 🗄️

We'll create an Elasticsearch index to store documents along with the generated vector embeddings. This will allow us to execute vector search when retrieving documents for our query.

Since we're using OpenAI's `text-embedding-ada-002` model, we need a 1536-dimensional [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field to store the embeddings.

In [ ]:
mappings = {
    "properties": {
        "text": { "type": "keyword" },
        "vector": {
            "type": "dense_vector",
            "dims": 1536,
            "index": True,
            "similarity": "cosine"
        }
    }
}

elasticsearch_client.indices.create(
    index="workplace-docs",
    mappings=mappings
)

## Load and process documents 📄

Time to load some data! We'll be using the workplace search example data, which is a list of employee documents and policies.

In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/workplace-search/example-data/data.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

print(f"Successfully loaded {len(workplace_docs)} documents")

Successfully loaded 15 documents


## Chunk documents into passages 🪓

As we're chatting with our bot, it will run semantic searches on the index to find the relevant documents. In order for this to be accurate, we need to split the full documents into small chunks (also called passages). This way the semantic search will find the passage within a document that most likely answers our question.

We'll use LangChain's `CharacterTextSplitter` and split the documents' text at 800 characters with some overlap between chunks.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append({
        "name": doc["name"],
        "summary": doc["summary"]
    })

text_splitter = CharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=400
)
docs = text_splitter.create_documents(content, metadatas=metadata)

print(f"Split {len(workplace_docs)} documents into {len(docs)} passages")

Created a chunk of size 866, which is longer than the specified 800
Created a chunk of size 1120, which is longer than the specified 800


Split 15 documents into 73 passages


Let's generate the embeddings and index the documents with them.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# Get the embeddings from openAI
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Extract page_content from the documents
texts = list(map(lambda t: t.page_content, docs))

text_embeddings = embeddings.embed_documents(texts)

print(f"Generated {len(text_embeddings)} embeddings")

# Persist the passage documents into Elasticsearch
actions = []
for i, passage in enumerate(docs):
    actions.append({"index": {}})
    actions.append({
        "text": passage.page_content,
        "vector": text_embeddings[i],
        "metadata": passage.metadata
    })

bulk_response = elasticsearch_client.bulk(
    operations=actions,
    index="workplace-docs"
)

print(f"Indexed {len(bulk_response['items'])} documents")

Generated 73 embeddings
Indexed 73 documents


## Chat with the chatbot 💬

Let's initialize our chatbot. We'll define Elasticsearch as a store for retrieving documents, OpenAI as the LLM to interpret questions and summarize answers, then we'll pass these to the conversational chain.

In [ ]:
from langchain.vectorstores.elastic_vector_search import ElasticKnnSearch
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

store = ElasticKnnSearch(
    es_connection=elasticsearch_client,
    index_name="workplace-docs",
    embedding=embeddings
)

retriever = store.as_retriever()

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

chat = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

Now we can ask questions from our chatbot!

See how the chat history is passed as context for each question.

In [ ]:
# Define a convenience function for Q&A
def ask(question, history):
    result = chat({"question": question, "chat_history": chat_history})
    print("QUESTION: ", question,
          "\nANSWER:  ", result["answer"],
          "\nSUPPORTING DOCUMENTS: ", list(map(lambda d: d.metadata["name"], list(result["source_documents"])))
    )
    history.append((question, result["answer"]))
    
chat_history = []

ask("What does NASA stand for?", chat_history)
ask("Which countries are part of it?", chat_history)
ask("Who are the team's leads?", chat_history)


QUESTION:  What does NASA stand for? 
ANSWER:    NASA stands for North America South America. 
SUPPORTING DOCUMENTS:  ['Sales Organization Overview', 'Code Of Conduct', 'Code Of Conduct', 'Swe Career Matrix']
QUESTION:  Which countries are part of it? 
ANSWER:    The North America South America region includes the United States, Canada, Mexico, as well as Central and South America. 
SUPPORTING DOCUMENTS:  ['Sales Organization Overview', 'Sales Organization Overview', 'Sales Organization Overview', 'Fy2024 Company Sales Strategy']
QUESTION:  Who are the team's leads? 
ANSWER:    Laura Martinez is the Area Vice-President of North America, and Gary Johnson is the Area Vice-President of South America. 
SUPPORTING DOCUMENTS:  ['Sales Organization Overview', 'Sales Organization Overview', 'Swe Career Matrix', 'Swe Career Matrix']


💡 _Try experimenting with other questions or after clearing the workplace data, and observe how the responses change._

# (Optional) Clean up 🧹

Once we're done, we can delete the Elasticsearch index.

In [ ]:
elasticsearch_client.indices.delete(index='workplace-docs')